In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
all_route_names=[]
all_route_links=[]

driver=webdriver.Chrome()
wait=WebDriverWait(driver,20)
driver.get("https://www.redbus.in/online-booking/kaac-transport")

def scrape_data():
    route_names=driver.find_elements(By.CLASS_NAME,"route")
    for route_name in route_names:
        all_route_names.append(route_name.text)

    route_links=driver.find_elements(By.CLASS_NAME,"route")
    hrefs=[route_link.get_attribute('href')for route_link in route_links]
    for href in hrefs:
        all_route_links.append(href)
           

#pagination
page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)
        time.sleep(4)
    scrape_data()
scrape_data()


# Close the browser after scraping
driver.quit()   
     




In [2]:
import pandas as pd
df=pd.DataFrame({"Route_names":all_route_names,"Route_links":all_route_links})


In [4]:
data=pd.DataFrame(df,columns=["Route_names","Route_links"])
print(data)

                    Route_names  \
0             Diphu to Guwahati   
1             Guwahati to Diphu   
2           Dokmoka to Guwahati   
3   Bokolia (assam) to Guwahati   
4   Guwahati to Bokolia (assam)   
5           Guwahati to Dokmoka   
6   Langhin (assam) to Guwahati   
7   Guwahati to Langhin (assam)   
8               Diphu to Hamren   
9     Manja (assam) to Guwahati   
10    Guwahati to Manja (assam)   
11              Hamren to Diphu   
12            Diphu to Jagiroad   
13           Guwahati to Hamren   
14    Guwahati to Manja (assam)   
15              Hamren to Diphu   
16            Diphu to Jagiroad   
17           Guwahati to Hamren   

                                          Route_links  
0   https://www.redbus.in/bus-tickets/diphu-to-guw...  
1   https://www.redbus.in/bus-tickets/guwahati-to-...  
2   https://www.redbus.in/bus-tickets/dokmoka-to-g...  
3   https://www.redbus.in/bus-tickets/bokolia-assa...  
4   https://www.redbus.in/bus-tickets/guwahati-to-... 

In [29]:
links=df['Route_links'].to_list()
print(links)

['https://www.redbus.in/bus-tickets/diphu-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-diphu', 'https://www.redbus.in/bus-tickets/dokmoka-to-guwahati', 'https://www.redbus.in/bus-tickets/bokolia-assam-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-bokolia-assam', 'https://www.redbus.in/bus-tickets/guwahati-to-dokmoka', 'https://www.redbus.in/bus-tickets/langhin-assam-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-langhin-assam', 'https://www.redbus.in/bus-tickets/diphu-to-hamren', 'https://www.redbus.in/bus-tickets/manja-assam-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-manja-assam', 'https://www.redbus.in/bus-tickets/hamren-to-diphu', 'https://www.redbus.in/bus-tickets/diphu-to-jagiroad', 'https://www.redbus.in/bus-tickets/guwahati-to-hamren', 'https://www.redbus.in/bus-tickets/guwahati-to-manja-assam', 'https://www.redbus.in/bus-tickets/hamren-to-diphu', 'https://www.redbus.in/bus-tickets/diphu-to-jagiroad', 'https://www.re

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import logging
import pandas as pd
import re

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Set up the driver
driver = webdriver.Chrome()

# List of links to scrape (replace with actual URLs)
links=['https://www.redbus.in/bus-tickets/diphu-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-diphu', 'https://www.redbus.in/bus-tickets/dokmoka-to-guwahati', 'https://www.redbus.in/bus-tickets/bokolia-assam-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-bokolia-assam', 'https://www.redbus.in/bus-tickets/guwahati-to-dokmoka', 'https://www.redbus.in/bus-tickets/langhin-assam-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-langhin-assam', 'https://www.redbus.in/bus-tickets/diphu-to-hamren', 'https://www.redbus.in/bus-tickets/manja-assam-to-guwahati', 'https://www.redbus.in/bus-tickets/guwahati-to-manja-assam', 'https://www.redbus.in/bus-tickets/hamren-to-diphu', 'https://www.redbus.in/bus-tickets/diphu-to-jagiroad', 'https://www.redbus.in/bus-tickets/guwahati-to-hamren', 'https://www.redbus.in/bus-tickets/guwahati-to-manja-assam', 'https://www.redbus.in/bus-tickets/hamren-to-diphu', 'https://www.redbus.in/bus-tickets/diphu-to-jagiroad', 'https://www.redbus.in/bus-tickets/guwahati-to-hamren']


# List to store all bus details
bus_data = []

# Function to scroll to the bottom of the page and wait for content to load
def scroll_to_bottom(driver):
    scroll_pause_time = 2  # Adjust this time if needed
    last_height = driver.execute_script("return document.body.scrollHeight")  # Get initial page height

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)  # Wait for page to load

        # Calculate new scroll height and compare with last height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit the loop if no new content is loaded
        last_height = new_height

# Loop through each link in the links list
for link in links:
    logging.info(f"Processing link: {link}")
    driver.get(link)  # Navigate to the link
    driver.maximize_window()
    time.sleep(5)  # Wait for the page to load

    # Locate multiple buttons if they exist and click them consecutively
    try:
        clickable_buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']")))

        for index, button in enumerate(clickable_buttons):
            driver.execute_script("arguments[0].click();", button)
            logging.info(f"Clicked button {index + 1} on {link}")
            time.sleep(3)  # Adjust time between clicks if needed

    except (NoSuchElementException, TimeoutException):
        logging.warning(f"Buttons not found or clickable for {link}, skipping click.")

    # Scroll down to the bottom of the page and wait for all content to load
    scroll_to_bottom(driver)

    # Locate the bus containers after all buttons have been clicked
    try:
        bus_container = driver.find_elements(By.CSS_SELECTOR, "div.clearfix.bus-item")

        # Extract details from each bus
        for bus in bus_container:
            bus_details = {
                "Bus_Name": None,
                "Bus_Type": None,
                "Departing_Time": None,
                "Duration": None,
                "Reaching_Time": None,
                "Star_Rating": None,
                "Price": None,
                "Seat_Availability": None
            }
            try:
                # bus_details["routename"]=link
                # bus_details["routelink"] = link
                bus_details["Bus_Name"] = bus.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                bus_details["Bus_Type"] = bus.find_element(By.XPATH, ".//div[contains(@class,'bus-type')]").text
                bus_details["Departing_Time"] = bus.find_element(By.XPATH, ".//div[contains(@class,'dp-time')]").text
                bus_details["Duration"] = bus.find_element(By.XPATH, ".//div[contains(@class,'dur')]").text
                bus_details["Reaching_Time"] = bus.find_element(By.XPATH, ".//div[contains(@class,'bp-time')]").text
                
                # Star rating extraction
                try:
                    star_rating_str = bus.find_element(By.XPATH, ".//div[contains(@class,'rating-sec')]").text
                    bus_details["Star_Rating"] = float(star_rating_str)  # Convert to float
                except NoSuchElementException:
                    bus_details["Star_Rating"] = None
                
                # Price extraction
                try:
                    price_str = bus.find_element(By.XPATH, "//span[contains(@class, 'f-19') and contains(@class, 'f-bold')]").text
                    bus_details["Price"] = float(re.sub(r'[^\d.]', '', price_str))  # Remove currency symbols and convert to float
                except NoSuchElementException:
                    bus_details["Price"] = None

                # Seat availability extraction
                try:
                    seats_info = bus.find_element(By.XPATH, ".//div[contains(@class,'seat-left')]").text
                    # Use regex to extract numbers from the seat information text
                    seats_match = re.search(r'\d+', seats_info)
                    if seats_match:
                        bus_details["Seat_Availability"] = int(seats_match.group())  # Convert to integer
                    else:
                        bus_details["Seat_Availability"] = "Unavailable"  # Handle cases where seat info isn't found
                except NoSuchElementException:
                    bus_details["Seat_Availability"] = "Not Available"
                bus_details["routename"]=link.split('/')[-1]
                bus_details["routelink"] = link    

            except NoSuchElementException:
                logging.warning(f"One or more elements not found in bus item on {link}.")

            # Append the bus details to the bus_data list
            bus_data.append(bus_details)

        logging.info(f"Collected bus details from link: {link}")

    except Exception as e:
        logging.error(f"An error occurred while collecting bus details from {link}: {e}")

# Create a DataFrame from the bus data
bus_df = pd.DataFrame(bus_data)

# Check if any data was collected
if not bus_data:
    logging.warning("No data was collected from any of the pages.")
else:
    # Print the DataFrame
    print(bus_df)

    # Optionally, save the DataFrame to a CSV file
    bus_df.to_csv("kaac.csv", index=False)

# Close the driver after all links have been processed
driver.quit()

2024-10-17 10:48:45,878 - Processing link: https://www.redbus.in/bus-tickets/diphu-to-guwahati
2024-10-17 10:49:02,188 - Buttons not found or clickable for https://www.redbus.in/bus-tickets/diphu-to-guwahati, skipping click.
2024-10-17 10:49:04,274 - Collected bus details from link: https://www.redbus.in/bus-tickets/diphu-to-guwahati
2024-10-17 10:49:04,274 - Processing link: https://www.redbus.in/bus-tickets/guwahati-to-diphu
2024-10-17 10:49:20,025 - Buttons not found or clickable for https://www.redbus.in/bus-tickets/guwahati-to-diphu, skipping click.
2024-10-17 10:49:24,333 - Collected bus details from link: https://www.redbus.in/bus-tickets/guwahati-to-diphu
2024-10-17 10:49:24,333 - Processing link: https://www.redbus.in/bus-tickets/dokmoka-to-guwahati
2024-10-17 10:49:40,237 - Buttons not found or clickable for https://www.redbus.in/bus-tickets/dokmoka-to-guwahati, skipping click.
2024-10-17 10:49:42,296 - Collected bus details from link: https://www.redbus.in/bus-tickets/dokmok

                   Bus_Name             Bus_Type Departing_Time Duration  \
0   KAAC TRANSPORT - 157250    Non AC Seater 2+2          11:30  06h 15m   
1            Kareng Travels  Non AC Seater (2+2)          11:45  06h 45m   
2   KAAC TRANSPORT - 157250    Non AC Seater 2+2          11:30  04h 15m   
3            Kareng Travels  Non AC Seater (2+2)          11:45  04h 45m   
4   KAAC TRANSPORT - 157250    Non AC Seater 2+2          11:30  03h 15m   
5            Kareng Travels  Non AC Seater (2+2)          11:45  04h 15m   
6   KAAC TRANSPORT - 157250    Non AC Seater 2+2          11:30  03h 30m   
7            Kareng Travels  Non AC Seater (2+2)          11:45  04h 30m   
8   KAAC TRANSPORT - 175492  Non AC Seater (2+2)          13:00  04h 30m   
9   KAAC TRANSPORT - 157250    Non AC Seater 2+2          11:30  05h 30m   
10           Kareng Travels  Non AC Seater (2+2)          11:45  06h 15m   
11  KAAC TRANSPORT - 157357  Non AC Seater (2+2)          12:00  06h 00m   
12  KAAC TRA

In [1]:
import pandas as pd
print(pd.__version__)


2.3.0


In [4]:
df1=pd.read_csv("apsrtc.csv")
df2=pd.read_csv("bsrtc.csv")
df3=pd.read_csv("hrtc.csv")
df4=pd.read_csv("kaac.csv")
df5=pd.read_csv("ksrtc.csv")
df6=pd.read_csv("KTCL.csv")
df7=pd.read_csv("pepsu.csv")
df8=pd.read_csv("rsrtc.csv")
df9=pd.read_csv("sbstc.csv")
df10=pd.read_csv("TSRTC.csv")
# added_df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis=0)
# added_df.to_csv("redbus.csv",index=False)

In [22]:
df1['state'] = 'Andhra Pradesh'
df2['state'] = 'Bihar'
df3['state'] = 'Himachal pradesh'
df4['state'] = 'Assam'
df5['state'] = 'Kerala'
df6['state'] = 'Goa'
df7['state'] = 'Punjab'
df8['state'] = 'Rajasthan'
df9['state'] = 'West Bengal[South Bengal]'
df10['state'] = 'Telengana'

In [23]:
added_df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis=0)
added_df.to_csv("redbus_full.csv",index=False)

In [24]:
import pandas as pd
df = pd.read_csv("redbus_full.csv")


In [27]:
df.dtypes

routename             object
routelink             object
Bus_Name              object
Bus_Type              object
Departing_Time        object
Duration              object
Reaching_Time         object
Star_Rating          float64
Price                float64
Seat_Availability      int64
state                 object
dtype: object

In [26]:
df.isna().sum()

routename             0
routelink             0
Bus_Name              0
Bus_Type              0
Departing_Time        0
Duration              0
Reaching_Time         0
Star_Rating          18
Price                 0
Seat_Availability     0
state                 0
dtype: int64

In [28]:
df[df['Star_Rating'].isna()]

,routename,routelink,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability,state
634,bangalore-to-tirupathi,https://www.redbus.in/bus-tickets/bangalore-to...,National travels,VE A/C Sleeper (2+1),18:00,06h 30m,00:30,NaN,306.0,31,Andhra Pradesh
1238,bangalore-to-ananthapur,https://www.redbus.in/bus-tickets/bangalore-to...,GOWTHAMI TOURS AND TRAVELS,Volvo Multi-Axle A/C (2+1),13:00,04h 30m,17:30,NaN,340.0,24,Andhra Pradesh
2533,rajahmundry-to-visakhapatnam,https://www.redbus.in/bus-tickets/rajahmundry-...,Sri Amaravathi Travels,NON A/C Hi-Tech Push Back (2+2),23:20,04h 40m,04:00,NaN,342.0,30,Andhra Pradesh
3574,vijayawada-to-visakhapatnam,https://www.redbus.in/bus-tickets/vijayawada-t...,Sri Amaravathi Travels,NON A/C Hi-Tech Push Back (2+2),19:00,09h 00m,04:00,NaN,605.0,30,Andhra Pradesh
5056,kozhikode-to-ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1498,Super Fast Non AC Seater (2+3),20:20,05h 45m,02:05,NaN,383.0,45,Kerala
5100,kozhikode-to-ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Cochin Express,Bharat Benz A/C Semi Sleeper (2+2),23:59,05h 56m,05:55,NaN,383.0,26,Kerala
5200,bangalore-to-kannur,https://www.redbus.in/bus-tickets/bangalore-to...,SAAM BUS,VE A/C Sleeper (2+1),21:45,07h 00m,04:45,NaN,568.0,19,Kerala
5279,kozhikode-to-thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1498,Super Fast Non AC Seater (2+3),20:20,11h 40m,08:00,NaN,723.0,46,Kerala
5496,kozhikode-to-thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1498,Super Fast Non AC Seater (2+3),20:20,03h 40m,00:00,NaN,181.0,45,Kerala
5755,kozhikode-to-thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1498,Super Fast Non AC Seater (2+3),20:20,03h 40m,00:00,NaN,181.0,45,Kerala


In [33]:
df['Star_Rating'] = df['Star_Rating'].fillna(df['Star_Rating'].mean())

In [34]:
df.isna().sum()

routename            0
routelink            0
Bus_Name             0
Bus_Type             0
Departing_Time       0
Duration             0
Reaching_Time        0
Star_Rating          0
Price                0
Seat_Availability    0
state                0
dtype: int64

MYSQL CONNECTION

In [37]:
import mysql.connector
mydb = mysql.connector.connect(
    host='localhost',
    user = 'root',
    password = '1234'

)
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE redbus")

In [38]:
df.dtypes

routename             object
routelink             object
Bus_Name              object
Bus_Type              object
Departing_Time        object
Duration              object
Reaching_Time         object
Star_Rating          float64
Price                float64
Seat_Availability      int64
state                 object
dtype: object

In [40]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="redbus"
)

mycursor = mydb.cursor()

mycursor.execute("""
CREATE TABLE busdetails (
  routename VARCHAR(255),
  routelink VARCHAR(255),
  Bus_Name VARCHAR(255),
  Bus_Type VARCHAR(255),
  Departing_Time VARCHAR(255),
  Duration VARCHAR(255),
  Reaching_Time VARCHAR(255),
  Star_Rating FLOAT,
  Price FLOAT,
  Seat_Availability BIGINT,
  state VARCHAR(255)
)
""")


In [41]:
import pandas as pd
from sqlalchemy import create_engine

# Replace these with your actual credentials
user = "root"
password = "1234"
host = "localhost"
database = "redbus"

# Create engine using SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Assuming your dataframe is named df
df.to_sql(name='busdetails', con=engine, if_exists='append', index=False)


8793